# Анализ данных о поездках на такси в Нью-Йорке

## Описание полей:

* pickup_dt – период с точностью до часа
* pickup_month – месяц
* borough – район Нью-Йорка, из которого был сделан заказ (5 районов + аэропорт)
* pickups – число поездок за период (час)
* hday – является ли день праздничным/выходным; Y — да,  N — нет
* spd – скорость ветра в милях в час
* vsb – видимость
* temp – температура в градусах Фаренгейта
* dewp – точка росы по Фаренгейту
* slp – давление
* pcp_01 – количество осадков за час
* pcp_06 – количество осадков за 6 часов
* pcp_24 – количество осадков за 24 часа
* sd – глубина снега в дюймах

Каждая строка в датасете — запись не об одной конкретной поездке, а о всех поездках, сделанных за определенный час из того или иного района Нью-Йорка, и о том, какие были показатели погоды в этот час.

## Цель: 
 1. Анализ данных о поездках на такси в Нью-Йорке;
 2. Получение инсайтов для бизнес-решений.

## Задачи:
1. Определить районы Нью-Йорка, где такси более и менее всего востребовано;
2. Определить районы, из которых по праздникам и выходным поступает больше всего заказов для коррекции количества машин в эти дни;
3. Определить сезонность востребованности такси.

In [20]:
import pandas as pd
taxi = pd.read_csv('taxi_nyc.csv')

### Предварительный просмотр данных.

In [21]:
taxi.head(10)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
5,2015-01-01 01:00:00,Jan,Staten Island,6,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
6,2015-01-01 01:00:00,Jan,NaN,4,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
7,2015-01-01 02:00:00,Jan,Bronx,120,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
8,2015-01-01 02:00:00,Jan,Brooklyn,1229,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
9,2015-01-01 02:00:00,Jan,EWR,0,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0


In [3]:
print('Количество строк: ', taxi.shape[0])
print('Количество столбцов: ', taxi.shape[1])

Количество строк:  29101
Количество столбцов:  14


### Проверка типов данных в столбцах датафрейма и определение преобладающего типа данных.

(Преобладает тип данных float64 (по 10 записей))

In [1]:
taxi.info()

NameError: name 'taxi' is not defined

### Приводим названия столбцов к единому формату.
В названиях трех столбцов встречается пробел: это столбцы `pcp 01`, `pcp 06` и `pcp 24`. 
Заменяем пробел на знак нижнего подчеркивания, чтобы было удобно обращаться к этим столбцам: через точку, без использования кавычек и скобок. Результат сохраняем в тот же датафрейм `taxi`.

In [23]:
taxi_renamed=taxi.rename(
columns={'pcp 01': 'pcp_01', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'})
taxi_renamed.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp_01,pcp_06,pcp_24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


### Проверка переименования.
Убеждаемся, что названия столбцов были успешно изменены и результат сохранен в датафрейм `taxi`, выведя обновлённый список названий столбцов.

In [24]:
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp 01', 'pcp 06', 'pcp 24', 'sd'],
      dtype='object')

### Проверяем частоту встречаемости районов Нью-Йорка. 
Для этого определяем число записей по каждому району Нью-Йорка.
(Районы в датафрейме встречаются одинаково часто)

In [25]:
taxi.borough.value_counts()

Brooklyn         4343
Bronx            4343
Manhattan        4343
Queens           4343
Staten Island    4343
EWR              4343
Name: borough, dtype: int64

### Общее количество поездок
Считаем общее количество поездок, чтобы узнать, насколько было востребовано такси в Нью-Йорке за период наблюдений. Для этого суммируем все значения в столбце `pickups`.

In [26]:
print('Количество всех поездок:', taxi.pickups.sum())

Количество всех поездок: 14265773


### Определяем район Нью-Йорка, где такси более востребовано.
Определяем район Нью-Йорка, из которого было совершено наибольшее количество поездок.
Такая информация может понадобиться, чтобы принять бизнес-решение о том, в каком районе увеличить количество машин такси.
(Такси более востребовано в Manhattan)

In [27]:
taxi.sort_values(by='pickups', ascending=False).head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
28604,2015-06-27 22:00:00,Jun,Manhattan,7883,N,8.00,5.5,60.00,56.50,1016.9,0.065000,0.00,0.00,0.0
28597,2015-06-27 21:00:00,Jun,Manhattan,7801,N,8.00,3.5,61.50,56.00,1017.0,0.045000,0.00,0.00,0.0
28611,2015-06-27 23:00:00,Jun,Manhattan,7711,N,10.25,3.2,60.75,57.25,1015.7,0.076667,0.01,0.00,0.0
28590,2015-06-27 20:00:00,Jun,Manhattan,7512,N,9.00,8.0,65.00,55.00,1018.2,0.010000,0.00,0.25,0.0
21739,2015-05-16 23:00:00,May,Manhattan,7271,N,0.00,9.1,71.00,60.00,1019.2,0.000000,0.00,0.00,0.0


### Определяем район Нью-Йорка, где такси востребовано меньше всего.
Для этого находим индекс района с наименьшим числом поездок. Сохраняем результат в переменную `min_pickups`
(Район, где такси востребовано менее всего - EWR)

In [28]:
taxi.groupby('borough').pickups.sum()
area_trip_counts = taxi.groupby('borough').pickups.sum()
min_pickups = area_trip_counts.idxmin()
print("Район с наименьшим числом поездок:", min_pickups)

Район с наименьшим числом поездок: EWR


### Определяем, из каких районов по праздникам и выходным дням в среднем поступает больше заказов, чем в обычные дни.

Такие расчеты помогут компании оптимизировать предложение машин такси и понять, в каких районах в выходные дни нужно дополнительно стимулировать водителей выходить на смену, чтобы машин хватало.

Группируем данные по району и типу дня (выходной/будний), рассчитываем среднее количество поездок для каждой группы.

Искомые районы - Bronx, Brooklyn 

In [30]:
taxi.groupby(['borough', 'hday'], as_index=False).pickups.mean()

,borough,hday,pickups
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
4,EWR,N,0.023467
5,EWR,Y,0.041916
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
8,Queens,N,308.899904
9,Queens,Y,320.730539


### Определяем число поездок в каждом районе по месяцам.

Эти расчеты помогут проследить динамику числа поездок, выявить закономерности и сезонность.  

Группируем данные по району и месяцу, считаем сумму поездок для каждой группы.


In [32]:
pickups_per_month=taxi.groupby(['borough', 'pickup_month'], as_index=False).pickups.sum()
pickups_by_mon_bor=pickups_per_month.sort_values('pickups', ascending=False)
pickups_by_mon_bor.head()

,borough,pickup_month,pickups
21,Manhattan,Jun,1995388
23,Manhattan,May,1888800
19,Manhattan,Feb,1718571
22,Manhattan,Mar,1661261
18,Manhattan,Apr,1648278


In [33]:
pickups_by_mon_bor.columns



Index(['borough', 'pickup_month', 'pickups'], dtype='object')

## Итоги

1. Район Нью-Йорка, где такси более востребовано - Manhattan;
менее всего востребовано - ;
2. Определить районы, из которых по праздникам и выходным поступает больше всего заказов для коррекции количества машин в эти дни;
3. Определить сезонность востребованности такси.






В проекте были проанализированны данные о поездках на такси в Нью-Йорке, получены инсайты для бизнес-решений.
Для выполнения проекта понадобились навыки работы с библиотекой pandas:

- Чтение данных в формате csv
- Определение размера датафрейма и типов данных в его столбцах
- Переименование колонок
- Группировка и агрегация данных
- Сортировка данных
- Получение индексов с минимальными и максимальными значениями
- Группировка по нескольким колонкам
- Сохранение результата в новый датафрейм